In [ ]:
### 3.5 图像分类数据集
# start coding at 01-23 10:30 on Mac
# target1: softmax的结构（输入和输出），矩阵表示
# target2: 实现softmax运算 X.sum(dim=0, keepdim=True) 的用法
# target3: gather函数的用法：取下标的值
# target4: 理解每次的输出epoch 1, loss 28395.907, train acc 0.840, test acc 0.830
#                      第几次？  全局loss         在训练集上的精度   在测试集上的精度

In [1]:
import torch
import torchvision
import numpy as np
import sys
sys.path.append("..") # 为了导入上层目录的d2lzh_pytorch
import d2lzh_pytorch as d2l

# 1. 获取数据

In [2]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

# 2. 初始化模型参数

In [3]:
num_inputs = 784
num_outputs = 10

W = torch.tensor(np.random.normal(0, 0.01, (num_inputs, num_outputs)), dtype=torch.float)
b = torch.zeros(num_outputs, dtype=torch.float)

## 需要模型梯度

In [4]:
W.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

# 3. 实现softmax运算

在介绍如何定义softmax回归之前，**我们先描述一下对如何对多维Tensor按维度操作**。
在下面的例子中，给定一个Tensor矩阵X。我们可以只对其中`同一列（dim=0）`或`同一行（dim=1）`的元素求和，并在结果中保留行和列这两个维度（keepdim=True）。

In [5]:
X = torch.tensor([
    [1,2,3],
    [4,5,6]])
print(X.sum(dim=0, keepdim=True))
print(X.sum(dim=1, keepdim=True))

tensor([[5, 7, 9]])
tensor([[ 6],
        [15]])


In [6]:
def softmax(X):
    X_exp = X.exp()
    partition = X_exp.sum(dim=1, keepdim=True)
    return X_exp / partition # 这里用了广播机制

# 4.定义模型

In [7]:
def net(X):
    return softmax(torch.mm(X.view(-1, num_inputs), W) + b)

# 5.定义损失函数

In [8]:
y_hat = torch.tensor([
[0.1, 0.3, 0.6],
[0.3, 0.2, 0.5]])

# 变量y是这2个样本的标签类别
y = torch.LongTensor([0, 2])

y_hat.gather(1, y.view(-1, 1))

tensor([[0.1000],
        [0.5000]])

In [11]:
def cross_entropy(y_hat, y):
    return - torch.log(y_hat.gather(1, y.view(-1,1)))

# 6.计算分类准确率

In [13]:
### 为了演示准确率的计算，下面定义准确率accuracy函数。
# 其中y_hat.argmax(dim=1)返回矩阵y_hat每行中最大元素的索引，且返回结果与变量y形状相同。
# 相等条件判断式(y_hat.argmax(dim=1) == y)是一个类型为ByteTensor的Tensor，
# 我们用float()将其转换为值为0（相等为假）或1（相等为真）的浮点型Tensor。

def accuracy(y_hat, y):
    return (y_hat.argmax(dim=1) == y).float().mean().item()

In [14]:
### 可以看到，第一个样本预测类别为2（该行最大元素0.6在本行的索引为2），与真实标签0不一致；
# 第二个样本预测类别为2（该行最大元素0.5在本行的索引为2），与真实标签2一致。
# 因此，这两个样本上的分类准确率为0.5。

print(accuracy(y_hat, y))

0.5


In [21]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
        
    return acc_sum / n

In [22]:
### 瞎猜
# 因为我们随机初始化了模型net，所以这个随机模型的准确率应该接近于类别个数10的倒数即0.1。
print(evaluate_accuracy(test_iter, net))

0.1065


In [36]:
num_epochs, lr = 5, 0.1

def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            y_hat = net(X)
            l = loss(y_hat, y).sum()
            
            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            
            # 根据损失函数求梯度
            l.backward()
            
            # 让优化器去优化参数
            if optimizer is None:
                d2l.sgd(params, lr, batch_size)
            else:
                optimizer.setp()
                
            # 全局损失
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).float().sum().item()
            n += y.shape[0]
        
        test_acc = evaluate_accuracy(test_iter, net)
        print("epoch %d, loss %.3f, train acc %.3f test acc %.3f"
             %(epoch+1, train_l_sum, train_acc_sum/n, test_acc))

In [37]:
train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [W, b], lr)

epoch 1, loss 28395.907, train acc 0.840 test acc 0.830
epoch 2, loss 27908.211, train acc 0.842 test acc 0.831
epoch 3, loss 27514.323, train acc 0.845 test acc 0.834
epoch 4, loss 27125.277, train acc 0.846 test acc 0.833
epoch 5, loss 26824.521, train acc 0.848 test acc 0.832


In [ ]:
# finished at 01-23 13;25